In [5]:
from src.network.client import Client

In [14]:
client = Client(port = 40910)
client.start()

Thank you for connecting


In [15]:
client.continue_loop = False

In [4]:
client.add_messages("test")

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/parfourator/miniconda3/envs/fdc-project/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/home/parfourator/dev/FDC-project/src/network/client.py", line 21, in run
    self.send_message()
  File "/home/parfourator/dev/FDC-project/src/network/client.py", line 35, in send_message
    self.socket.send(message.encode())
BrokenPipeError: [Errno 32] Broken pipe



message sent : test
